# **Village Libraries Case Study**

### **Import Packages**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### **Connect and Migrate Data From Google Sheets**

In [5]:
import gspread
import pandas as pd
from google.auth import default
from google.colab import auth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open_by_key('1LKHCtYcqtjkgqBRpRYNC7KCR-xmLCcvQve3BLbnsJzI')
worksheet = spreadsheet.get_worksheet(0)
df = pd.DataFrame(worksheet.get())
# Code for making the first row as header. Remove if not needed.
df.columns = df.iloc[0]
df = df.drop(0)
df.head()

,computer_reservation_id,session_time,library_id,student_attended,mentor_attended,student_id,student_gender,grade_level,family_status,family_support_level,graduation_obstacle
1,195,8/18/2022,2,FALSE,TRUE,46,Male,10,Staying with both parents,2,Financial
2,908,8/18/2022,2,FALSE,TRUE,49,Female,9,Staying with both parents,1,Financial
3,51,8/25/2022,2,TRUE,TRUE,45,Female,10,Staying with both parents,2,Financial
4,10839,9/15/2022,3,FALSE,TRUE,177,Male,4,Staying with an extended family member,3,Financial
5,198,9/8/2022,2,TRUE,TRUE,46,Male,10,Staying with both parents,2,Financial


In [6]:
df.describe()

,computer_reservation_id,session_time,library_id,student_attended,mentor_attended,student_id,student_gender,grade_level,family_status,family_support_level,graduation_obstacle
count,8736,8736,8736,8736,8736,8736,8736,8736,8736,8736,8701
unique,8736,628,16,2,2,563,4,14,6,6,9
top,195,2/11/2023,8,TRUE,TRUE,NULL,Female,5,Staying with both parents,2,Financial
freq,1,44,1221,6368,6101,544,4726,1331,4590,3068,4264


In [7]:
df.columns.drop(['session_time', 'student_attended', 'mentor_attended', 'student_gender', 'family_status', 'graduation_obstacle'])

Index(['computer_reservation_id', 'library_id', 'student_id', 'grade_level',
       'family_support_level'],
      dtype='object', name=0)

### **Data Processing**

In [50]:
# make a copy of the original dataset
df_cleaned = df.copy()
# drop na columns
df_cleaned[df.columns] = df_cleaned[df.columns].replace('', np.nan)
df_cleaned = df_cleaned.dropna()
# change session_time to date time
df_cleaned['session_time'] = pd.to_datetime(df_cleaned['session_time'])
# change object columns to numeric
cols = df_cleaned.columns.drop(['session_time', 'student_attended', 'mentor_attended', 'student_gender', 'family_status', 'graduation_obstacle'])
for col in cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
# re-code binary columns
mapping = {'TRUE': 1, 'FALSE': 0}
df_cleaned[['student_attended', 'mentor_attended']] = df_cleaned[['student_attended', 'mentor_attended']].applymap(lambda x: mapping[x])


# create few more columns
df_groupby = df_cleaned.groupby('student_id').agg(
    student_attendance_rate=('student_attended', 'mean'),
    mentor_attendance_rate=('mentor_attended', 'mean'),
    enrollment_length=('session_time', lambda x: (x.max() - x.min()).days)
).reset_index()

df_cleaned = df_cleaned.merge(df_groupby, on = 'student_id', how = 'left')

<ipython-input-50-8ae01542d55f>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-8ae01542d55f>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
df_cleaned.sort_values(by = ['student_id'], ascending = True).head()

,computer_reservation_id,session_time,library_id,student_attended,mentor_attended,student_id,student_gender,grade_level,family_status,family_support_level,graduation_obstacle,student_attendance_rate,mentor_attendance_rate,enrollment_length
56,65,2022-12-01,2,1,1,45.0,Female,10.0,Staying with both parents,2.0,Financial,0.846154,0.846154,189 days
24,53,2022-09-08,2,1,1,45.0,Female,10.0,Staying with both parents,2.0,Financial,0.846154,0.846154,189 days
25,49,2022-08-11,2,1,1,45.0,Female,10.0,Staying with both parents,2.0,Financial,0.846154,0.846154,189 days
38,74,2023-02-02,2,1,1,45.0,Female,10.0,Staying with both parents,2.0,Financial,0.846154,0.846154,189 days
13,50,2022-08-18,2,1,1,45.0,Female,10.0,Staying with both parents,2.0,Financial,0.846154,0.846154,189 days


In [10]:
df_cleaned.dtypes

computer_reservation_id              int64
session_time                datetime64[ns]
library_id                           int64
student_attended                     int64
mentor_attended                      int64
student_id                         float64
student_gender                      object
grade_level                        float64
family_status                       object
family_support_level               float64
graduation_obstacle                 object
student_attendance_rate            float64
mentor_attendance_rate             float64
enrollment_length          timedelta64[ns]
dtype: object

### **Exploratory Analysis**

#### 1, Gender Analysis

In [52]:
def plot_factor_vs_attendance(df, factor):
    df_plot = df.groupby(factor)['student_attendance_rate'].mean().reset_index()

    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Count of Students by {factor.capitalize()}", f"Student Attendance Rate by {factor.capitalize()}"))

    count_data = df[factor].value_counts().reset_index()
    count_data.columns = [factor, 'count']

    unique_values = df[factor].nunique()
    color_palette = px.colors.qualitative.Safe[:unique_values]

    fig.add_trace(
        go.Bar(x=count_data[factor], y=count_data['count'], marker_color=color_palette, text=count_data['count'], textposition='outside'),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=df_plot[factor], y=df_plot['student_attendance_rate'], marker_color=color_palette, text=df_plot['student_attendance_rate'].round(2), textposition='outside'),
        row=1, col=2
    )

    fig.update_layout(height=600, width=1200, title_text=f"{factor.capitalize()} Analysis", showlegend=False)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Student Attendance Rate", row=1, col=2)

    fig.show()

plot_factor_vs_attendance(df_cleaned, 'student_gender')

#### 2, Across Different Libraries

In [53]:
plot_factor_vs_attendance(df_cleaned, 'library_id')

#### 3, Grade Level Difference

In [54]:
plot_factor_vs_attendance(df_cleaned, 'grade_level')

####4, Family Status

In [55]:
plot_factor_vs_attendance(df_cleaned, 'family_status')

#### 5, Family Support Level

In [56]:
plot_factor_vs_attendance(df_cleaned, 'family_support_level')

#### 6, Graduation Obstacles

In [57]:
plot_factor_vs_attendance(df_cleaned, 'graduation_obstacle')

#### 7, Mentor Attendance Rate

In [58]:
fig = px.scatter(
    df_groupby,
    x='mentor_attendance_rate',
    y='student_attendance_rate',
    title="Effect of Mentor Attendance Rate on Student Attendance Rate",
    labels={'mentor_attendance_rate': 'Mentor Attendance Rate', 'student_attendance_rate': 'Student Attendance Rate'},
    trendline='lowess',
    height=800,
    width=1200
)
fig.show()

#### 8, Enrollment Length

In [59]:
fig = px.scatter(
    df_groupby,
    x='enrollment_length',
    y='student_attendance_rate',
    title="Effect of Length of Enrollment on Student Attendance Rate",
    labels={'enrollment_length': 'Enrollment Length', 'student_attendance_rate': 'Student Attendance Rate'},
    trendline='lowess',
    height=800,
    width=1200
)
fig.show()

### **Predictive Analysis**

#### 1, Model Selection

In [62]:
# Define numerical and categorical columns
numerical_cols = [ 'library_id', 'student_attended', 'mentor_attended', 'student_id', 'grade_level', 'family_support_level', 'mentor_attendance_rate']
categorical_cols = ['student_gender', 'family_status', 'graduation_obstacle', 'enrollment_length']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

# Split data into training and test sets
X = df_cleaned.drop('student_attendance_rate', axis=1)
y = df_cleaned['student_attendance_rate']
y = y.fillna(y.median())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, X_train, X_test, y_train, y_test):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return mae, mse, rmse, r2

# Evaluate all models
results = {}
for model_name, model in models.items():
    results[model_name] = evaluate_model(model, X_train, X_test, y_train, y_test)
results_df = pd.DataFrame(results, index=['MAE', 'MSE', 'RMSE', 'R2']).T
results_df


,MAE,MSE,RMSE,R2
Linear Regression,0.090051,0.017135,0.130903,0.709879
Gradient Boosting,0.088802,0.014693,0.121216,0.751228
XGBoost,0.031301,0.003004,0.054811,0.949135
Random Forest,0.007232,0.001417,0.037643,0.976009


#### 2, Random Forest

In [33]:
df_cleaned = df_cleaned.dropna(subset=['student_attendance_rate'])
# Define numerical and categorical columns
numerical_cols = [ 'library_id', 'student_attended', 'mentor_attended', 'student_id', 'grade_level', 'family_support_level', 'mentor_attendance_rate']
categorical_cols = ['student_gender', 'family_status', 'graduation_obstacle', 'enrollment_length']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Split data into training and test sets
X = df_cleaned.drop('student_attendance_rate', axis=1)
y = df_cleaned['student_attendance_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = RandomForestRegressor(random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)


# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

MAE: 0.01076279873330066
MSE: 0.0028088390896834916
RMSE: 0.052998481956405996
R^2: 0.9576670022120015


In [42]:
# Get feature importances
feature_importances = pipeline.named_steps['model'].feature_importances_
feature_names = numerical_cols + list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols))
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)

threshold = 0.01
significant_features_df = feature_importance_df[feature_importance_df['Importance'] > threshold].sort_values(by='Importance', ascending=True)

# Plot the significant feature importances using Plotly
fig_importances = px.bar(significant_features_df, x='Importance', y='Feature', title='Significant Feature Importances', orientation='h')
fig_importances.update_layout(xaxis_title='Importance', yaxis_title='Feature', height=600)
fig_importances.show()

In [43]:
# Create a DataFrame with actual and predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results_df['Residual'] = results_df['Actual'] - results_df['Predicted']

# Plot the predicted vs actual values
fig_results = px.scatter(results_df, x='Actual', y='Predicted', title='Predicted vs Actual Values')
fig_results.add_shape(type='line', x0=results_df['Actual'].min(), y0=results_df['Actual'].min(), x1=results_df['Actual'].max(), y1=results_df['Actual'].max(), line=dict(color='red', dash='dash'))
fig_results.update_layout(xaxis_title='Actual', yaxis_title='Predicted', height=600)
fig_results.show()

In [44]:
# Plot the residuals
fig_residuals = px.scatter(results_df, x='Predicted', y='Residual', title='Residual Plot')
fig_residuals.update_layout(xaxis_title='Predicted', yaxis_title='Residual', height=600)
fig_residuals.add_shape(type='line', x0=results_df['Predicted'].min(), y0=0, x1=results_df['Predicted'].max(), y1=0, line=dict(color='red', dash='dash'))
fig_residuals.show()

### **Summary**

#### 1. Gender Analysis
- Female students have a slightly higher attendance rate (0.73) compared to male students (0.71), and there are more female students (4716) than male students (3289).
- Focus on understanding and addressing the specific needs and challenges of male students.

#### 2. For Different Libraries
- Library 1, 10, and 20 have the highest student attendance rates (0.91, 0.89, and 0.89 respectively), while Library  20 has the lowest attendance rate (0.21).
- Understand and replicate the successful practices of Library IDs 1, 10, and 20 in other libraries to improve overall student attendance rates.

#### 3. Grade Levels
- Students in grade level 1 have the highest attendance rate (0.84), while students in grade level 12 have the lowest attendance rate (0.35). The number of students is highest in grade levels 6, 5, and 4.
- Implement targeted interventions and support for students in higher grade levels, particularly grade level 12, to help improve their attendance rates.

#### 4. Family Status
- Students staying with both parents have the highest attendance rate (0.77), followed by students staying with an extended family member (0.70) and students from single-parent families (0.67). The lowest attendance rate is among students with unknown family status (0.24).
- Provide additional support and resources to students with unknown family status and those staying with single parents to help improve their attendance rates. This could include mentoring programs, counseling, and family engagement initiatives.

#### 5. Family Support Level
- Students with the highest family support level (4) have the highest attendance rate (0.81), while students with the lowest family support level (1) have the lowest attendance rate (0.67).
- Develop programs and initiatives to engage families with lower support levels, providing resources and support to help them better support their children's education and attendance.

#### 6. Graduation Obstacles
- Students facing "Required to work" as a graduation obstacle have the highest attendance rate (0.91), while those with "NULL" obstacles have the lowest attendance rate (0.24). Financial obstacles are the most common, but their attendance rate (0.68) is relatively high.
- Implement targeted support programs to assist students with unknown (NULL) obstacles to improve their attendance rates. Additionally, continue to support students with financial obstacles, as they represent a large portion of the student population and have a moderate attendance rate.

#### 7. Mentor Attendance Rate
- We observed a positive correlation between mentor attendance rate and student attendance rate, indicating that higher mentor attendance is associated with higher student attendance.
- Encourage and support mentors to maintain high attendance rates, as this appears to positively influence student attendance. Consider implementing policies or programs that promote consistent mentor involvement.

#### 8. Length of Enrollment
- Student attendance rates tend to drop when the enrollment period hits between 100 to 200 days but become more steady and tend to improve as the enrollment length increases beyond this period.
- Implement targeted interventions around the 100 to 200-day mark to address and mitigate the drop in attendance, and continue to provide support to help improve attendance rates over longer enrollment periods.










#### Model Selection
- Based on the evaluation metrics, the Random Forest model outperformed all other models with the lowest MAE, MSE, RMSE, and the highest R² score, making it the best choice for predicting the student attendance rate.

#### Model Performance
- The Predicted vs. Actual Values plot shows that the predictions align closely with the actual values along the red dashed line, indicating a high accuracy of the Random Forest model in predicting student attendance rates.
- The residual plot shows that the residuals are randomly scattered around the horizontal axis, indicating that the Random Forest model has no obvious patterns of errors and provides a good fit for predicting student attendance rates.

#### Important Features
- Aside from the student's attendance history, the feature importance plot indicates that the most significant factors influencing student attendance rate are whether the mentor attended the sessions, other influential factors include the specific library, grade level, and family support.



